# Développer une application web dynamique cartographique avec Panel (*ou comment construire un jeu interactif ?*)

***METTRE LE LIEN VERS LE JEU***

| Sujet | Auteur | Dernière Màj |
|:---:|:------:|:------------:|
| Application webmapping | Thomas LEYSENS (*IR AME*) | *30 novembre 2023* |


## Enjeux & Propositions
| Enjeu | Proposition |
|:------|:------------|
| Global | Développer une application de webmapping (*découvrir les possibilités de la librairie Python Panel*) |
| Classe et fonctions Python | Explications et exemples |
| Structurer une application | Widgets, figures et callbacks |
| Utiliser des paramètres externes | Charger et lire un fichier YAML |
| Utiliser des données externes | Charger et lire un fichier CSV avec Pandas et Geopandas |
| Contraintes d'une application en ligne | Différences entre local et en ligne |

```python
#READ & lOAD YAML PARAMETERS
url = "https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_params.yml"
r = requests.get(url)
params = yaml.safe_load(r.content)
RANGES = params["ranges"]
BUFFER_VALUE = params["buffer_value"]
NB_Q = params["nb_questions"]
CSV_SEP = ";"
CSV_FILE = "https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_QA.csv"
```

```cmd
panel convert game_panel.ipynb --to pyodide-worker --out pyodide
```

```cmd
python3 -m http.server
```

http://localhost:8000/pyodide/game_panel.html

## Préparation
### Données

Dans notre cas, nous souhaitons construire un fichier de questions/réponses au format CSV (*comma-separated values*) afin qu'il soit chargé dans notre application:
* c'est un jeu cartographique, nous allons donc récupérer des données sur [OpenStreetMap](https://www.openstreetmap.org/)
* nous souhaitons agrémenter les réponses d'un lien Wikipédia (*quand il est disponible*), un prétraitement est donc nécessaire

#### OpenStreetMap

> ***AVERTISSEMENT: Veillez à bien respecter les conditions d'utilisation des données open source d'OpenStreetMap, cf. [License Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl)***

Allons chercher quelques lieux symboliques de Le Croisic sur [OpenStreetMap](https://www.openstreetmap.org/), voici comment faire pour l'exemple en question (*Pierre Longue*):

![OSM query](../img/game/OSM.png)

#### CSV

Petit à petit, nous pouvons construire notre [fichier CSV](https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_QA.csv) de questions/réponses avec tous les détails souhaités, dont voici un extrait ci-dessous:

| question | answer | hint | x | y | wikidata |
|:---------|:-------|:-----|--:|--:|:---------|
| My eye, when opened, can save the ships. Who am I ? | Phare du Tréhic | Search in the North | -2.5237 | 47.3082 | Q20971192 |
| If you break your leg, where do you go ? | Ancien hôpital du Croisic | Search for the red circle with a cross | -2.51493 | 47.2918 | Q21418896 |
| I've got wheat and I want flour, where do I have to go ? | Moulin de Beauvran | Search near south of Laboratoire de biologie marine | -2.51796 | 47.2855 | Q21152416 |
| I'm BotMan, where do I park my Botmobile ? | Manoir de Kervaudu | It is a manor surrounded by green areas | -2.5264  | 47.2964 | Q3286209 |

On peut constater qu'il nous manque les URL des pages Wikipedia (*elles sont parfois présentes dans les données OSM mais nous préférons utiliser le Wikidata car il est plus souvent renseigné*). Il est possible de les retrouver via l'identifiant [Wikidata](https://fr.wikipedia.org/wiki/Wikidata). Plutôt que de faire des recherches manuelles, nous allons créer une méthode Python pour récupérer automatiquement l'information et compléter le fichier CSV (*le fichier Python se trouve [ici](https://github.com/thomleysens/Tutoriels_AME/blob/main/tutoriels/utils.py) et nous allons le détailler et l'expliquer*). 

Tout d'abord, voici le code brut:

```python
"""
@author: thomleysens
"""

import requests
import pandas as pd


def get_wiki_url(
    wikidata_id,
    base_url="https://www.wikidata.org/wiki/Special:EntityData/{}.json",
    wiki="frwiki"
):
    """
    Get wikipedia url from wikidata ID

    Parameters
    ----------
    wikidata_id (str): Wikidata ID
    base_url (str): Url to request Entity and get JSON response
                    Default: "https://www.wikidata.org/wiki/Special:EntityData/{}.json
    wiki (str): Wiki reference
                Default: "frwiki"

    Returns
    -------
    url (str):
        - Wikipedia URL if wikidata_id is not NA
        - None if wikidata_id is NA
    """
    if pd.isna(wikidata_id) is False:
        url = requests.get(
            base_url.format(wikidata_id)
        ).json()["entities"][wikidata_id]["sitelinks"][wiki]["url"]
    else:
        url = None

    return url
```

In [ ]:
import pandas as pd

from utils import get_wiki_url

csv_file = "game_QA.csv"
df = pd.read_csv(
    csv_file,
    sep=";",
    encoding="utf-8"
)
df["url"] = df["wikidata"].map(
    get_wiki_url
)
df.to_csv(
    csv_file,
    sep=";",
    encoding="utf-8",
    index=False
)

## Licenses librairies Python / Données / API

### Données & API

| API / Données | License / Droit d'usage | Documentation |
|:--------------|:------------------------|:--------------|
| [OpenStreetMap](https://www.openstreetmap.org/) | [License Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl/) | [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) |

### Librairies Python

Voir le [README](https://github.com/thomleysens/Tutoriels_AME/tree/main#librairies-python-utilis%C3%A9es-dans-ce-d%C3%A9p%C3%B4t).

## Liens utiles

* [Librairie Python Panel](https://panel.holoviz.org/)
* [Convertir une application Panel pour la mise en ligne sur GitHub Pages](https://panel.holoviz.org/how_to/wasm/convert.html)
* [OpenStreetMap (*données cartographiques open source*)](https://www.openstreetmap.org/)
* [Construire un JupyterBook](https://jupyterbook.org/en/stable/start/your-first-book.html)